<a href="https://colab.research.google.com/github/mjgalaz/Supermarket-ETL-Web-scraping/blob/main/Web_Scraping_Wikipedia_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Web scraping wikipedia con BeautifulSoup

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# URL de la página web
url = 'https://en.wikipedia.org/wiki/List_of_supermarket_chains'

# Obtener el HTML de la página
response = requests.get(url)
html_content = response.content

# Utilizar BeautifulSoup para procesar el HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Encontrar la tabla de interés
table = soup.find('table', {'class': 'wikitable'})

# Reemplazar etiquetas <br> por comas en la columna 'Headquarters' antes de leer la tabla con Pandas
for br in table.find_all('br'):
    br.replace_with(', ')

# Leer la tabla en un DataFrame de Pandas
tabla_multinational = pd.read_html(str(table))[0]

# Eliminar la columna 'Map'
tabla_multinational.drop(columns=['Map'], inplace=True)

# Mostrar el DataFrame
display(tabla_multinational)




,Company,Headquarters,Served countries (besides the headquarters),Number of locations,Number of employees
0,7-Eleven,"Japan, United States","Australia, Canada, China, Cambodia, Denmark, H...",83485,170000
1,Aeon,Japan,"Australia, Cambodia, China, Hong Kong, India, ...",20008,"560,000+"
2,Ahold Delhaize,Netherlands,"Belgium (as Albert Heijn and Delhaize), Czech ...",7659,375000
3,Aldi Nord,Germany,"Belgium, Denmark, France, Luxembourg, Netherla...",5241,72811
4,Aldi Süd,Germany,"Australia, Austria (as Hofer), China, Hungary,...",7178,201361
...,...,...,...,...,...
360,Robinsons Supermarket,Philippines,NaN,274,NaN
361,SM Retail,Philippines,NaN,NaN,NaN
362,citysuper,Hong Kong,"Taiwan, Mainland China",41,NaN
363,Kansai Super,Japan,NaN,60,NaN


Copia tabla obtenida

In [2]:
# Crear una copia del DataFrame
copia_tabla_multinational = tabla_multinational.copy()

# Mostrar la copia del DataFrame
display(copia_tabla_multinational)


,Company,Headquarters,Served countries (besides the headquarters),Number of locations,Number of employees
0,7-Eleven,"Japan, United States","Australia, Canada, China, Cambodia, Denmark, H...",83485,170000
1,Aeon,Japan,"Australia, Cambodia, China, Hong Kong, India, ...",20008,"560,000+"
2,Ahold Delhaize,Netherlands,"Belgium (as Albert Heijn and Delhaize), Czech ...",7659,375000
3,Aldi Nord,Germany,"Belgium, Denmark, France, Luxembourg, Netherla...",5241,72811
4,Aldi Süd,Germany,"Australia, Austria (as Hofer), China, Hungary,...",7178,201361
...,...,...,...,...,...
360,Robinsons Supermarket,Philippines,NaN,274,NaN
361,SM Retail,Philippines,NaN,NaN,NaN
362,citysuper,Hong Kong,"Taiwan, Mainland China",41,NaN
363,Kansai Super,Japan,NaN,60,NaN


In [3]:
# Agrupar variables numéricas
numeric_cols = ['Number of locations', 'Number of employees']

# Eliminar texto entre paréntesis en la columna 'Served countries (besides the headquarters)'
copia_tabla_multinational['Served countries (besides the headquarters)'] = copia_tabla_multinational['Served countries (besides the headquarters)'].str.replace(r'\(.*?\)', '', regex=True)

# Eliminar texto entre paréntesis en la columna 'Headquarters'
copia_tabla_multinational['Headquarters'] = copia_tabla_multinational['Headquarters'].str.replace(r'\(.*?\)', '', regex=True)

# Reemplazar 'USA' por 'United States' en la columna 'Served countries (besides the headquarters)'
copia_tabla_multinational['Served countries (besides the headquarters)'] = copia_tabla_multinational['Served countries (besides the headquarters)'].str.replace('USA', 'United States')

import re
# Definir patrón de búsqueda para encontrar 'and' que no esté seguido por 'Bosnia and Herzegovina'
patron = r'\b(?!Bosnia\s+and\s+Herzegovina\b)(and)\b'

# Aplicar reemplazo en la columna 'Served countries (besides the headquarters)'
copia_tabla_multinational['Served countries (besides the headquarters)'] = copia_tabla_multinational['Served countries (besides the headquarters)'].str.replace(patron, ',', regex=True)

# Limpiar la variable 'Company'
copia_tabla_multinational['Company'] = copia_tabla_multinational['Company'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
copia_tabla_multinational['Company'] = copia_tabla_multinational['Company'].str.lower()

# Limpiar la variable 'Headquarters'
copia_tabla_multinational['Headquarters'] = copia_tabla_multinational['Headquarters'].str.lower()

# Limpiar la variable 'Served countries (besides the headquarters)'
copia_tabla_multinational['Served countries (besides the headquarters)'] = copia_tabla_multinational['Served countries (besides the headquarters)'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
copia_tabla_multinational['Served countries (besides the headquarters)'] = copia_tabla_multinational['Served countries (besides the headquarters)'].str.lower()

# Eliminar el signo "+" y la coma de las columnas en numeric_cols
for col in numeric_cols:
    copia_tabla_multinational[col] = copia_tabla_multinational[col].str.replace('[\+,]', '', regex=True)

# Eliminar todo después del primer número en las columnas en numeric_cols
for col in numeric_cols:
    copia_tabla_multinational[col] = copia_tabla_multinational[col].str.replace(r'^(\d+).*', r'\1', regex=True)

from unicodedata import normalize
# Función para normalizar datos con Unicode para luego quitar los espacios usando .replace().
# Argumentos de entrada: Nombre de columna o lista con nombres de columnas.
# Retorna: columna o columnas sin espacios en blanco
def remove_whitespace(x):

    if isinstance(x, str):
        return normalize('NFKC', x).replace(' ', '')
    else:
        return x

# Aplicar función remove_whitespace a columnas en variable y las reemplazamos en la copia
copia_tabla_multinational[numeric_cols] = copia_tabla_multinational[numeric_cols].applymap(remove_whitespace)

# Verificamos si se quitaron los espacios en blanco
copia_tabla_multinational.head()

# Mostrar tipo de datos de la copia
copia_tabla_multinational.dtypes

# Asignar el tipo de dato numérico a columnas en la copia
copia_tabla_multinational[numeric_cols] = copia_tabla_multinational[numeric_cols].apply(pd.to_numeric)

# Convertir las columnas 'Company', 'Headquarters' y 'Served countries (besides the headquarters)' al tipo de dato string
copia_tabla_multinational['Company'] = copia_tabla_multinational['Company'].astype('string')
copia_tabla_multinational['Headquarters'] = copia_tabla_multinational['Headquarters'].astype('string')
copia_tabla_multinational['Served countries (besides the headquarters)'] = copia_tabla_multinational['Served countries (besides the headquarters)'].astype('string')

# Ver tabla
display(copia_tabla_multinational)

# Verificamos que las columnas con números tengan el tipo de dato numérico asignado en la copia
copia_tabla_multinational.dtypes


,Company,Headquarters,Served countries (besides the headquarters),Number of locations,Number of employees
0,7eleven,"japan, united states",australia canada china cambodia denmark hong k...,83485.0,170000.0
1,aeon,japan,australia cambodia china hong kong india indon...,20008.0,560000.0
2,ahold delhaize,netherlands,belgium czech republic greece indonesia ne...,7659.0,375000.0
3,aldi nord,germany,belgium denmark france luxembourg netherlands ...,5241.0,72811.0
4,aldi sd,germany,australia austria china hungary ireland italy...,7178.0,201361.0
...,...,...,...,...,...
360,robinsons supermarket,philippines,<NA>,274.0,NaN
361,sm retail,philippines,<NA>,NaN,NaN
362,citysuper,hong kong,taiwan mainland china,41.0,NaN
363,kansai super,japan,<NA>,60.0,NaN


Company                                         string
Headquarters                                    string
Served countries (besides the headquarters)     string
Number of locations                            float64
Number of employees                            float64
dtype: object

Nueva variable countries

In [4]:
import pandas as pd

# Suponiendo que tienes un DataFrame llamado copia_tabla_multinational con las columnas 'Headquarters' y 'Served countries (besides the headquarters)'

# Llenar los valores NA con una cadena vacía en las columnas 'Headquarters' y 'Served countries (besides the headquarters)'
copia_tabla_multinational['Headquarters'].fillna('', inplace=True)
copia_tabla_multinational['Served countries (besides the headquarters)'].fillna('', inplace=True)

# Concatenar las columnas 'Headquarters' y 'Served countries (besides the headquarters)'
copia_tabla_multinational['countries'] = copia_tabla_multinational['Headquarters'].str.cat(copia_tabla_multinational['Served countries (besides the headquarters)'], sep=', ')

# Eliminar la coma inicial y final si está presente
copia_tabla_multinational['countries'] = copia_tabla_multinational['countries'].apply(lambda x: x.strip(', '))

# Mostrar el DataFrame con la nueva columna 'countries'
display(copia_tabla_multinational)


,Company,Headquarters,Served countries (besides the headquarters),Number of locations,Number of employees,countries
0,7eleven,"japan, united states",australia canada china cambodia denmark hong k...,83485.0,170000.0,"japan, united states, australia canada china c..."
1,aeon,japan,australia cambodia china hong kong india indon...,20008.0,560000.0,"japan, australia cambodia china hong kong indi..."
2,ahold delhaize,netherlands,belgium czech republic greece indonesia ne...,7659.0,375000.0,"netherlands, belgium czech republic greece ..."
3,aldi nord,germany,belgium denmark france luxembourg netherlands ...,5241.0,72811.0,"germany, belgium denmark france luxembourg net..."
4,aldi sd,germany,australia austria china hungary ireland italy...,7178.0,201361.0,"germany, australia austria china hungary irel..."
...,...,...,...,...,...,...
360,robinsons supermarket,philippines,,274.0,NaN,philippines
361,sm retail,philippines,,NaN,NaN,philippines
362,citysuper,hong kong,taiwan mainland china,41.0,NaN,"hong kong, taiwan mainland china"
363,kansai super,japan,,60.0,NaN,japan


Tabla final

In [5]:
# Seleccionar las columnas deseadas
tabla_multinational_final = copia_tabla_multinational[['Company','countries','Number of locations', 'Number of employees']].copy()

# Mostrar el DataFrame final
display(tabla_multinational_final)

,Company,countries,Number of locations,Number of employees
0,7eleven,"japan, united states, australia canada china c...",83485.0,170000.0
1,aeon,"japan, australia cambodia china hong kong indi...",20008.0,560000.0
2,ahold delhaize,"netherlands, belgium czech republic greece ...",7659.0,375000.0
3,aldi nord,"germany, belgium denmark france luxembourg net...",5241.0,72811.0
4,aldi sd,"germany, australia austria china hungary irel...",7178.0,201361.0
...,...,...,...,...
360,robinsons supermarket,philippines,274.0,NaN
361,sm retail,philippines,NaN,NaN
362,citysuper,"hong kong, taiwan mainland china",41.0,NaN
363,kansai super,japan,60.0,NaN


Guardar tabla final en formato csv

In [8]:
# Guarda el DataFrame en un archivo CSV
tabla_multinational_final.to_csv('tabla_multinational_final.csv')

# Lee el archivo para verificar su creación
pd.read_csv('tabla_multinational_final.csv')


,Unnamed: 0,Company,countries,Number of locations,Number of employees
0,0,7eleven,"japan, united states, australia canada china c...",83485.0,170000.0
1,1,aeon,"japan, australia cambodia china hong kong indi...",20008.0,560000.0
2,2,ahold delhaize,"netherlands, belgium czech republic greece ...",7659.0,375000.0
3,3,aldi nord,"germany, belgium denmark france luxembourg net...",5241.0,72811.0
4,4,aldi sd,"germany, australia austria china hungary irel...",7178.0,201361.0
...,...,...,...,...,...
360,360,robinsons supermarket,philippines,274.0,NaN
361,361,sm retail,philippines,NaN,NaN
362,362,citysuper,"hong kong, taiwan mainland china",41.0,NaN
363,363,kansai super,japan,60.0,NaN


Guardar tabla final en formato csv en el ordenador

In [10]:
# Cargar o descargar archivos
from google.colab import files

# Descarga archivo con datos de tabla
files.download("tabla_multinational_final.csv")

print('Listo, en un momento saldrá la opción "Guardar Como" para descargar el archivo...')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Listo, en un momento saldrá la opción "Guardar Como" para descargar el archivo...
